In [2]:
import tensorflow as tf
import tensorboard

from tensorflow.keras import layers, Sequential

import string
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

pd.read_csv('../data/Emotion_final.csv')

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [ ]:
le = LabelEncoder()